In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

**Importing dataset**

In [87]:
df=pd.read_csv("/content/processed_insurance_dataset.csv")
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,0,16884.92400
1,18,1,33.770,1,0,1,1725.55230
2,28,1,33.000,3,0,1,4449.46200
3,33,1,22.705,0,0,2,21984.47061
4,32,1,28.880,0,0,2,3866.85520


In [88]:
df.describe()

,age,sex,bmi,children,smoker,region,charges
count,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,0.505232,30.663397,1.094918,0.204783,1.484305,13270.422265
std,14.049960,0.500160,6.098187,1.205493,0.403694,1.104885,12110.011237
min,18.000000,0.000000,15.960000,0.000000,0.000000,0.000000,1121.873900
25%,27.000000,0.000000,26.296250,0.000000,0.000000,1.000000,4740.287150
50%,39.000000,1.000000,30.400000,1.000000,0.000000,1.000000,9382.033000
75%,51.000000,1.000000,34.693750,2.000000,0.000000,2.000000,16639.912515
max,64.000000,1.000000,53.130000,5.000000,1.000000,3.000000,63770.428010


Selecting feature and target

In [100]:
X = df[['smoker', 'age', 'bmi']].values
y = df['charges'].values


Shuffle and Train test split

In [101]:
indices = np.arange(len(X))
np.random.shuffle(indices)

X = X[indices]
y = y[indices]

split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


Normalizing training data

In [102]:
X_mean = X_train.mean(axis=0)
X_std = X_train.std(axis=0)

X_train = (X_train - X_mean) / X_std
X_test = (X_test - X_mean) / X_std


Scaling target variable

In [103]:
y_mean = y_train.mean()
y_std = y_train.std()

y_train_scaled = (y_train - y_mean) / y_std


Adding bias term

In [104]:
X_train = np.c_[np.ones(len(X_train)), X_train]
X_test = np.c_[np.ones(len(X_test)), X_test]


Ridge regression

In [105]:
lambda_ridge = 0.1
weights_ridge = np.zeros(X_train.shape[1])

lr = 0.01
epochs = 4000
n = len(y_train_scaled)

for _ in range(epochs):
    y_pred = X_train.dot(weights_ridge)
    error = y_pred - y_train_scaled

    gradients = (2/n) * X_train.T.dot(error) + 2 * lambda_ridge * weights_ridge
    weights_ridge -= lr * gradients


In [106]:
y_test_pred_scaled = X_test.dot(weights_ridge)
y_test_pred_ridge = y_test_pred_scaled * y_std + y_mean


Evaluation

In [107]:
mse_ridge = np.mean((y_test - y_test_pred_ridge)**2)
rmse_ridge = np.sqrt(mse_ridge)
r2_ridge = 1 - np.sum((y_test - y_test_pred_ridge)**2) / np.sum((y_test - np.mean(y_test))**2)

print("RIDGE → R2:", r2_ridge)


RIDGE → R2: 0.7278734810621992


Lasso Regression

In [108]:
lambda_lasso = 0.05
weights_lasso = np.zeros(X_train.shape[1])

for _ in range(epochs):
    y_pred = X_train.dot(weights_lasso)
    error = y_pred - y_train_scaled

    gradients = (2/n) * X_train.T.dot(error) + lambda_lasso * np.sign(weights_lasso)
    weights_lasso -= lr * gradients


Testing

In [109]:
y_test_pred_scaled = X_test.dot(weights_lasso)
y_test_pred_lasso = y_test_pred_scaled * y_std + y_mean


In [110]:
mse_lasso = np.mean((y_test - y_test_pred_lasso)**2)
rmse_lasso = np.sqrt(mse_lasso)
r2_lasso = 1 - np.sum((y_test - y_test_pred_lasso)**2) / np.sum((y_test - np.mean(y_test))**2)

print("LASSO → R2:", r2_lasso)


LASSO → R2: 0.733414425847656


In [111]:
def predict_regularized(smoker, age, bmi, model="ridge"):
    X_new = np.array([[smoker, age, bmi]])
    X_new = (X_new - X_mean) / X_std
    X_new = np.c_[np.ones(X_new.shape[0]), X_new]

    if model == "ridge":
        y_scaled = X_new.dot(weights_ridge)
    else:
        y_scaled = X_new.dot(weights_lasso)

    return y_scaled * y_std + y_mean


In [112]:
print("Ridge:", predict_regularized(1, 45, 32, "ridge"))
print("Lasso:", predict_regularized(1, 45, 32, "lasso"))


Ridge: [32157.50591543]
Lasso: [33288.95395519]
